# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.**

In [125]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn import metrics
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [126]:
df = pd.read_csv('data/Advertising.zip')
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [127]:
df.isnull().sum()

Unnamed: 0    0
TV            0
radio         0
newspaper     0
sales         0
dtype: int64

Пропусков нет

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [128]:
# признак unnamed:0 игнорируем, он дублирует индексы со сдвигом на 1

X = df[['TV', 'radio', 'newspaper']].values
y = df['sales'].values

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [129]:
# добавляем в матрицу наблюдений единичный столбец
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
# преобразуем вектор y в массив
y = y.reshape(-1, 1)

# проверим размерность данных
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [130]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}w_j$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [131]:
# задаем функцию для решения задания
def coordinate_descent(X,y, w, num_iters = 1000):
    '''
    Функция реализует алгоритм координатного спуска.
    Параметры функции:
        X: матрица наблюдений
        y: целевая переменная
        w: вектор коэффициентов
        num_iters: кол-во итераций
    Функция возвращает:
        w: вектор с коэффициентами
        mse: значение метрики MSE
        mae: значение метрики MAE
    '''
    mse = []
    mae = []
    for i in range(num_iters):
        for k in range(m):
            h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])     # Вычисляем прогноз
            w[k] =  (X[:,k].T @ (y - h))                        # Обновляем k-й коэффициент
            mse = sum((y - (X @ w)) ** 2)/(len(y))              # Вычисляем MSE
            mae = sum(abs(y - (X @ w)))/(len(y))                # Вычисляем MAE
    return w, mse, mae

m = X.shape[1]
n = len(y)
w = np.zeros((4,1))

w, cost_mse, mae = coordinate_descent(X,y, w)

print(f'Коэффициенты линейной регрессии: \n {w}')
print(f'MSE: {cost_mse}')
print(f'MAE: {mae}')

Коэффициенты линейной регрессии: 
 [[ 41.56217205]
 [110.13144155]
 [ 73.52860638]
 [ -0.55006384]]
MSE: [2.78412631]
MAE: [1.25201123]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [132]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_predict = model.predict(X)

model_mse = metrics.mean_squared_error(y,y_predict)
model_mae = metrics.mean_absolute_error(y,y_predict)
 
print(f'Коэффициенты модели: \n {model.coef_}')
print(f'MSE: {model_mse}')
print(f'MAE: {model_mae}')

Коэффициенты модели: 
 [[ 41.56217205 110.13144155  73.52860638  -0.55006384]]
MSE: 2.784126314510936
MAE: 1.2520112296870662


Если вы все сделали верно, они должны практически совпасть!

Коэффициенты и метрики в обоих случаях совпадают

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [133]:
X = df[['TV', 'radio', 'newspaper']].values
y = df['sales'].values

X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

**Добавим единичный столбец**

In [134]:
# добавляем в матрицу наблюдений единичный столбец
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
w = np.zeros([4, 1])

print(X.shape, y.shape)

(200, 4) (200, 1)


**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [135]:
def mse_error(y, y_predict):
    '''
    Функция вычисляет среднеквадратичную ошибку MSE.
    Параметры функции:
        y: массив/вектор с реальными значениями целевой переменной
        y_predict: массив/вектор с предсказанными значениями целевой переменной
    Функция возвращает:
        значение среднеквадратичной ошибки MSE
    '''
    mse = sum((y - y_predict) ** 2)/(len(y))
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [136]:
y_predict = np.mean(y, axis=0)

print(mse_error(y, y_predict))

[27.08574375]


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [137]:
def lin_pred(X, w):
    '''
    Функция вычисляет массив прогнозов
    Параметры функции:
        X: матрица наблюдений
        w: вектор коэффициентов линейной модели
    Функция возвращает:
        массив прогнозов
    '''
    y_predict = X@w
    y_predict = y_predict.reshape(-1, 1)

    return y_predict

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [138]:
def stoch_grad_step(X, y, w, train_index, eta):
    '''
    Функция реализует шаг стохастического градиентного спуска
    Параметры функции:
        X: матрица наблюдений
        y: массив реальных значений целевой переменной
        w: вектор коэффициентов линейной модели
        train_index: индекс объекта обучающей выборки
        eta: шаг градиентного спуска
    Функция возвращает:
        обновленные коэффициенты линейной модели
    '''
    object = X[train_index]
    y_predict = lin_pred(X[train_index], w)                     # предсказываем целевую переменную
    direction = (object*2*(y_predict - y[train_index])) / 1     # рассчитываем направление изменения весов модели по формуле
    w_new = w - direction*eta                                   # обновляем веса(коэффициенты) модели

    return w_new[0]

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [139]:
def stochastic_gradient_descent(X, y, w, eta, max_iter, min_weight_dist, seed):
    '''
    Функция реализует стохастический градиентный спуск
    Параметры функции:
        X: матрица наблюдений
        y: массив реальных значений целевой переменной
        w: вектор коэффициентов линейной модели
        eta: параметр, определяющий темп обучения
        max_iter: максимальное число итераций
        min_weight_dist: минимальное расстояние между векторами весов
        seed: для воспроизводимости эксперимета
    Функция возвращает:
        w: обновленные коэффициенты линейной модели
        errors: MSE
    '''
    weight_dist = float("inf") # задаем начальное значение расстояния между векторами - бесконечность
    errors = []
    iter_num = 0

    np.random.seed(seed)

    # выполняем пока расстояние между векторами весов больше ограничения и не превысили количества итераций
    while weight_dist > min_weight_dist and iter_num < max_iter:
        random_index = np.random.randint(X.shape[0])                            # задаем случайный индекс строки матрицы X
        new_w = stoch_grad_step(X, y, w, random_index, eta)                     # обновляем коэффициены
        error = mse_error(y[random_index], lin_pred(X[random_index], new_w))    # MSE
        errors.append(error)
        weight_dist = np.linalg.norm(w-new_w)                                   # расстояние между векторами
        w = new_w
        iter_num += 1

    return w, errors

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [140]:
# реализуем стохастический градиентный спуск
w, errors = stochastic_gradient_descent(X, y, w, eta=0.0001, max_iter=1e5, min_weight_dist=1e-8, seed=42)

**Выведите вектор весов, к которому сошелся метод.**

In [141]:
w

array([14.01943426,  3.92231475,  2.82193654, -0.0326486 ])

**Выведите среднеквадратичную ошибку на последней итерации.**

In [142]:
errors[-1][0]

1.0730595722999264e-11

In [143]:
print(f'MSE: {mse_error(y,lin_pred(X,w))}')
print(f'MAE: {sum(abs(y - lin_pred(X,w)))/(len(y))}')

MSE: [2.78493236]
MAE: [1.25089949]


Для проверки полученных результатов выполним градиентный стохастический спуск, используя библиотеку sklearn

In [144]:
reg = SGDRegressor(max_iter=100000)
reg.fit(X,y)

y_predict = reg.predict(X)

# вычисляем метрики MSE и MAE
model_mse = metrics.mean_squared_error(y,y_predict)
model_mae = metrics.mean_absolute_error(y,y_predict)

# выводим полученные данные
print(f'Коэффициенты модели: \n {reg.coef_}')
print(f'MSE: {model_mse}')
print(f'MAE: {model_mae}')

Коэффициенты модели: 
 [ 7.01050789  3.91559302  2.77040546 -0.00990973]
MSE: 2.7845883108149714
MAE: 1.2529978806525406


Был реализован алгоритм стохастического градиентного спуска